In [0]:
%pip install keplergl 

In [0]:
# After %pip installs on Databricks, it's good practice to restart Python
dbutils.library.restartPython()


In [0]:
# Run the geo_utils notebook
%run ./geo_utils.ipynb

In [0]:
%run ../config

In [0]:

db = 'predictive_maintenance_navy_test'
spark.sql(f'USE CATALOG {catalog}')
spark.sql(f'USE SCHEMA {db}')

In [0]:
# ---- Platforms ---
platforms_sdf = spark.read.option("header", True).csv('/Volumes/public_sector/predictive_maintenance_navy_test/raw_landing/PdM_Platform_Data - navy_platform_data_underway_sim.csv')

platforms_sdf = platforms_sdf.drop('lat', 'long')
platforms_sdf = platforms_sdf.withColumnRenamed('sim_lat', 'lat').withColumnRenamed('sim_lon', 'long')

# platforms_sdf = spark.table('ship_meta')


from pyspark.sql.types import DoubleType

platforms_sdf = platforms_sdf.withColumn("lat", platforms_sdf["lat"].cast(DoubleType()))
platforms_sdf = platforms_sdf.withColumn("long", platforms_sdf["long"].cast(DoubleType()))
platforms_pdf = spiderfy_duplicates(platforms_sdf.toPandas(), lon_col='long', radius_m=400)
platforms_sdf = spark.createDataFrame(platforms_pdf)


# ---- USER CONFIG ----
# --- Stock and routing table --- 
STOCK_TABLE     = "public_sector.predictive_maintenance_navy_test.parts_silver"  # e.g., "maritime.stock_locations"
ROUTES_TABLE    = "public_sector.predictive_maintenance_navy_test.shipment_recommendations"           # e.g., "maritime.routes"

from pyspark.sql import functions as F

# --- Load Delta tables
stock_sdf = spark.table(STOCK_TABLE)
stock_sdf = stock_sdf.select("stock_location", "stock_location_id", "lat", "long").distinct()
stock_sdf = stock_sdf.withColumn("lat", F.round(F.col("lat"), 0)).withColumn("long", F.round(F.col("long"), 0))


routes_sdf = spark.table(ROUTES_TABLE)
routes_sdf = routes_sdf.filter(F.col("qty_shipped") > 0)

In [0]:
display(routes_sdf)

In [0]:

# --- Build arcs: join source (stock) and target (platform) coordinates
arcs_sdf = (
    routes_sdf.alias("r")
    .join(
        stock_sdf.select(
            F.col("stock_location_id").alias("stock_location_id_s"),
            F.col("lat").alias("source_lat"),
            F.col("long").alias("source_lng"),
            F.col("stock_location").alias("stock_name")
        ),
        F.col("r.stock_location_id") == F.col("stock_location_id_s"),
        "left"
    )
    .join(
        platforms_sdf.select(
            F.col("designator").alias("designator_p"),
            F.col("lat").alias("target_lat"),
            F.col("long").alias("target_lng"),
            # F.col("name").alias("platform_name")
        ),
        F.col("r.designator") == F.col("designator_p"),
        "left"
    )
    .select(
        F.col("r.stock_location_id"),
        F.col("r.designator"),
        F.col("r.type"),
        F.col("r.qty_shipped"),
        "source_lat","source_lng","target_lat","target_lng",
        # F.col("r.priority").alias("priority"),
        "stock_name"
    )
)


In [0]:
import pandas as pd
from keplergl import KeplerGl

platforms_pdf = platforms_sdf.toPandas()
stock_pdf     = stock_sdf.toPandas()
arcs_pdf      = arcs_sdf.toPandas()

def make_kepler_config():
    return {
        "version": "v1",
        "config": {
            "mapState": {
                "latitude": 34.5,
                "longitude": -112.0,
                "zoom": 2.5,
                "pitch": 45,
                "bearing": 0
            },
            "mapStyle": {
                #"styleType": "dark",
                "visibleLayerGroups": {"label": True, "road": True, "border": False, "building": False, "water": True, "land": True}
            },
            "visState": {
                "filters": [],
                "interactionConfig": {
                    "tooltip": {
                        "fieldsToShow": {
                            "platforms": [{"name": "designator"}],
                            "stock_locations": [{"name": "stock_location_id"}, {"name": "stock_location"}],
                            "shipping_routes": [{"name": "stock_name"}, {"name": "designator"}, {"name": "type"}],
                        },
                        "compareMode": False,
                        "enabled": True
                    }
                },
                "layerBlending": "normal",
                "layers": [
                    {
                        "id": "platforms-layer",
                        "type": "point",
                        "config": {
                            "dataId": "platforms",
                            "label": "Platforms",
                            "color": [255, 140, 0],
                            "columns": {"lat": "lat", "lng": "long"},
                            "isVisible": True,
                            "visConfig": {
                                "radius": 50,
                                "opacity": 0.9,
                                "outline": True,
                                "thickness": 3
                            }
                        }
                    },
                    {
                        "id": "stock-layer",
                        "type": "point",
                        "config": {
                            "dataId": "stock_locations",
                            "label": "Stock Locations",
                            "color": [0, 204, 255],
                            "columns": {"lat": "lat", "lng": "long"},
                            "isVisible": True,
                            "visConfig": {
                                "radius": 50,
                                "opacity": 0.9,
                                "outline": True,
                                "thickness": 3
                            }
                        }
                    },
                    {
                        "id": "routes-layer",
                        "type": "arc",
                        "config": {
                            "dataId": "shipping_routes",
                            "label": "Shipping Routes",
                            "color": [60, 220, 130],
                            "columns": {
                                "lat0": "source_lat",
                                "lng0": "source_lng",
                                "lat1": "target_lat",
                                "lng1": "target_lng"
                            },
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.6,
                                "thickness": 3,
                                "colorRange": {
                                    "name": "ColorBrewer YlGn-6",
                                    "type": "sequential",
                                    "category": "ColorBrewer",
                                    "colors": ["#f7fcb9", "#d9f0a3", "#addd8e", "#78c679", "#41ab5d", "#238443"]
                                },
                                "sizeRange": [1, 15]
                            }
                        }
                    }
                ]
            }
        }
    }

config = make_kepler_config()
m = KeplerGl(config=config)
m.add_data(data=platforms_pdf, name="platforms")
m.add_data(data=stock_pdf, name="stock_locations")
m.add_data(data=arcs_pdf, name="shipping_routes")

m


In [0]:
map_path = '/Volumes/public_sector/predictive_maintenance_navy/raw_landing/map_test.html'
m.save_to_html(file_name=map_path, read_only=False)